In [1]:
import pandas as pd

In [2]:
covar_path = '/share/garg/311_data/sb2377/clean_codebase/tract_demographics.csv'
base_data_path = '/share/garg/311_data/sb2377/clean_codebase/three_year_base.csv'
reporting_data_path = '/share/garg/311_data/sb2377/clean_codebase/three_year_reports_full.h5'
save_path = '/share/garg/311_data/sb2377/clean_codebase/three_year_full.h5'
types = {'street': {'typeagency': 'StreetConditionDOT',
                    'rating_path': '/share/garg/311_data/sb2377/clean_codebase/processed_streets_three_year.h5'},
         'park': {'typeagency': 'MaintenanceorFacilityDPR',
                  'rating_path': '/share/garg/311_data/sb2377/clean_codebase/processed_parks_three_year.h5'},
         'rodent': {'typeagency': 'RodentDOHMH',
                    'rating_path': '/share/garg/311_data/sb2377/clean_codebase/processed_rodents_three_year.h5'},
         'restaurant': {'typeagency': 'FoodDOHMH',
                        'rating_path': '/share/garg/311_data/sb2377/clean_codebase/processed_restaurant_three_year.h5'},
         'dcwp': {'typeagency': 'ConsumerComplaintDCWP',
                  'rating_path': '/share/garg/311_data/sb2377/clean_codebase/processed_dcwp_three_year.h5'}}

In [3]:
cols_to_save = ['GEOID', 
                'report_week', 
                'report_week_of_year',
                'normalized_rating', 
                'finegrained_reported', 
                'real_rating_observed', 
                'typeagency',
                'type_idxs',
                'node_idxs',
                'rating_week',
                'rating_week_of_year',
                'finegrained_id'
               ]

In [4]:
covariates_arr = pd.read_csv(covar_path)
base_df = pd.read_csv(base_data_path)
reporting_df = pd.read_hdf(reporting_data_path, 'df')
dfs = {}
for t in types:
    dfs[t] = pd.read_hdf(types[t]['rating_path'], 'df')

In [5]:
# get type indices
type_df = base_df[['typeagency', 'type_idxs']].drop_duplicates()
indices = {}
for t in types:
    indices[t] = type_df[type_df['typeagency'] == types[t]['typeagency']]['type_idxs'].iloc[0]

In [6]:
# set null values in reporting data for rating specific columns
reporting_df['normalized_rating'] = 0
reporting_df['finegrained_reported'] = reporting_df['reported']
reporting_df['real_rating_observed'] = 0

In [7]:
# for each complaint type with observed ratings, combine data with and without ratings
full_dfs = []

for t in types:
    type_df = dfs[t]
    type_idx = indices[t]
    
    # get data with ratings
    # only include data with valid geoids
    valid_geoid = set(base_df['GEOID'].unique())
    type_rating_df = type_df[type_df['GEOID'].isin(valid_geoid)]

    # add node_idxs
    type_rating_df = pd.merge(type_rating_df, base_df[['node_idxs', 'GEOID']].drop_duplicates(), on='GEOID')
    type_rating_df = type_rating_df[cols_to_save]

    # get data without ratings
    type_no_rating_df = reporting_df[reporting_df['type_idxs'] == type_idx]
    all_geoids = set(type_no_rating_df['GEOID'].unique())
    rating_geoids = set(type_rating_df['GEOID'].unique())
    no_rating_geoids = all_geoids.difference(rating_geoids)
    type_no_rating_df = type_no_rating_df[type_no_rating_df['GEOID'].isin(no_rating_geoids)]
    type_no_rating_df['rating_week'] = -1
    type_no_rating_df['rating_week_of_year'] = -1
    type_no_rating_df['finegrained_id'] = -1
    type_no_rating_df = type_no_rating_df[cols_to_save]

    # combine
    full_type_df = pd.concat([type_rating_df, type_no_rating_df])
    full_dfs.append(full_type_df)

In [8]:
# get data for types with observed ratings
rating_observed_df = pd.concat(full_dfs)
rating_observed_df['type_rating_observed'] = 1

# get data for types with unobserved ratings
all_types = set(base_df['type_idxs'].unique())
rating_observed_types = set(rating_observed_df['type_idxs'].unique())
rating_unobserved_types = all_types.difference(rating_observed_types)
rating_unobserved_df = reporting_df[reporting_df['type_idxs'].isin(rating_unobserved_types)]
rating_unobserved_df['rating_week'] = -1
rating_unobserved_df['rating_week_of_year'] = -1
rating_unobserved_df['finegrained_id'] = -1
rating_unobserved_df = rating_unobserved_df[cols_to_save]
rating_unobserved_df['type_rating_observed'] = 0

# combine
full_df = pd.concat([rating_observed_df, rating_unobserved_df])

# get covariates
full_df = pd.merge(full_df, covariates_arr, on='GEOID')

/share/pierson/tmp_directory_location_please_read_readme/sb2377_tmp/ipykernel_776400/24203610.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_unobserved_df['rating_week'] = -1
/share/pierson/tmp_directory_location_please_read_readme/sb2377_tmp/ipykernel_776400/24203610.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_unobserved_df['rating_week_of_year'] = -1
/share/pierson/tmp_directory_location_please_read_readme/sb2377_tmp/ipykernel_776400/24203610.py:12: SettingWithCopyWarning: 
A 

: 

: 

: 

In [1]:
assert(len(full_df[full_df.isna().any(axis=1)]) == 0)

NameError: name 'full_df' is not defined

In [10]:
full_df.to_hdf(save_path, key='df', mode='w')

/share/pierson/tmp_directory_location_please_read_readme/sb2377_tmp/ipykernel_4093353/4163754240.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['typeagency', 'finegrained_id'], dtype='object')]

  full_df.to_hdf(save_path, key='df', mode='w')
